In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch_geometric
!pip install sklearn

In [2]:
import pandas as pd
import requests
from time import sleep
import os
import json
from geopy.distance import geodesic
import sys
import pickle
import numpy as np
import torch_geometric
import torch_geometric.nn as nn
import torch_geometric.utils as utils
import torch_geometric.transforms as T
from torch_geometric.data import hetero_data
import torch
from sklearn.preprocessing import MultiLabelBinarizer

import numpy as np
import torch
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import Sequential, Linear
from torch.nn import ReLU
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.nn.conv import HeteroConv, SAGEConv, GATConv, GraphConv
from torch_geometric.loader import NeighborLoader, HGTLoader
import torch.nn.functional as F
import torch_sparse

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/ml_proj/'

data_path = folder_path + 'data/'

In [4]:
listings = pd.read_csv(folder_path + 'data/listings_clean.csv')
cta = pd.read_csv(folder_path + 'data/cta.csv')
pois = pd.read_csv(folder_path + 'data/pois.csv')

In [5]:
listings_price = listings['price']
listings_features = listings.drop(columns=['price'])

In [ ]:
poi_edge_index = [[], []]
poi_edge_weights = []


if os.path.exists(folder_path + 'data/poi_edges_dist.pkl'):
    with open(folder_path + 'data/poi_edges_dist.pkl', 'rb') as file:
        (poi_edge_index, poi_edge_weights) = pickle.load(file)
else:
    for listing_id, listing in listings.iterrows():
        listing_coord = listing[['latitude', 'longitude']].values * (np.pi / 180) * 6371000

        poi_coords = pois[['lat', 'lon']].values * (np.pi / 180) * 6371000

        distances = np.linalg.norm((listing_coord - poi_coords).astype(int), axis=1)
        weights = 1 / (distances + .01)
        #weights = distances

        # d < thresh => 1/d > 1/thresh
        thresh = 50

        close_pois = np.where(distances < thresh)[0]

        if close_pois.size > 0:
            poi_edge_index[0].extend([listing_id] * close_pois.size)
            poi_edge_index[1].extend(close_pois)
            poi_edge_weights.extend(weights[close_pois])

        if listing_id % 1000 == 0:
            print(f"{listing_id}/{len(listings)}")

    with open(folder_path + 'data/poi_edges_dist.pkl', 'wb') as file:
        pickle.dump((poi_edge_index, poi_edge_weights), file)

print(len(poi_edge_index[0]))
print(len(poi_edge_index[1]))
print(len(poi_edge_weights))
print(poi_edge_index[0][:20])
print(poi_edge_index[1][:20])
print(poi_edge_weights[:20])

In [ ]:
cta_edge_index = [[], []]
cta_edge_weights = []


if os.path.exists(folder_path + 'data/cta_edges_dist.pkl') and False:
    with open(folder_path + 'data/cta_edges_dist.pkl', 'rb') as file:
        (cta_edge_index, cta_edge_weights) = pickle.load(file)
else:
    for listing_id, listing in listings.iterrows():
        listing_coord = listing[['latitude', 'longitude']].values * (np.pi / 180) * 6371000

        cta_coords = cta[['Latitude', 'Longitude']].values * (np.pi / 180) * 6371000

        distances = np.linalg.norm((listing_coord - cta_coords).astype(int), axis=1)
        weights = 1/distances

        thresh = 300

        close_cta = np.where(weights >1/ thresh)[0]

        if close_cta.size > 0:
            cta_edge_index[0].extend([listing_id] * close_cta.size)
            cta_edge_index[1].extend(close_cta)
            cta_edge_weights.extend(weights[close_cta])


        if listing_id % 1000 == 0:
            print(f"{listing_id}/{len(listings)}")

    with open(folder_path + 'data/cta_edges_dist.pkl', 'wb') as file:
        pickle.dump((cta_edge_index, cta_edge_weights), file)

print(len(cta_edge_index[0]))
print(len(cta_edge_index[1]))
print(len(cta_edge_weights))
print(cta_edge_index[0][:20])
print(cta_edge_index[1][:20])
print(cta_edge_weights[:20])

In [ ]:
listings_edge_index = [[], []]
listings_edge_weights = []

with open(folder_path + 'data/doc_inference.pkl', 'rb') as file:
        doc_inference = pickle.load(file)

doc_inference_t = torch.tensor(doc_inference)

if os.path.exists(folder_path + 'data/listings_edges.pkl') and False:
    with open(folder_path + 'data/listings_edges.pkl', 'rb') as file:
        (listings_edge_index, listings_edge_weights) = pickle.load(file)
else:
    for listing_id, listing in listings.iterrows():
        listing_coord = listing[['latitude', 'longitude']].values * (np.pi / 180) * 6371000

        embedding_i = doc_inference[listing_id]

        embedding_i_expanded = doc_inference_t[listing_id].unsqueeze(0).expand_as(doc_inference_t)

        cosine_similarity = F.cosine_similarity(embedding_i_expanded, doc_inference_t, dim=1)
        cosine_similarity[listing_id] = 0


        listings_coord = listings[['latitude', 'longitude']].values * (np.pi / 180) * 6371000

        distances = np.linalg.norm((listing_coord - listings_coord).astype(int), axis=1)
        distances[listing_id] = 99999
        weights = 1 / (distances + .01)

        # d < thresh => 1/d > 1/thresh
        thresh = 100
        cosine_sim_thresh = .3

        sim_listings = np.where(torch.tensor(distances < thresh) & (cosine_similarity > cosine_sim_thresh))[0]

        if sim_listings.size > 0:
            listings_edge_index[0].extend([listing_id] * sim_listings.size)
            listings_edge_index[1].extend(sim_listings)
            listings_edge_weights.extend(weights[sim_listings])

        if listing_id % 1000 == 0:
            print(f"{listing_id}/{len(listings)}")

    #with open(folder_path + 'data/listings_edges.pkl', 'wb') as file:
        #pickle.dump((listings_edge_index, listings_edge_weights), file)

print(len(listings_edge_index[0]))
print(len(listings_edge_index[1]))
print(len(listings_edge_weights))
print(listings_edge_index[0][:20])
print(listings_edge_index[1][:20])
print(listings_edge_weights[:20])

In [23]:
from sklearn.preprocessing import StandardScaler

data = HeteroData()

scaler = StandardScaler()
listings_scaled = scaler.fit_transform(listings.to_numpy())
pois_scaled = scaler.fit_transform(pois[['price', 'type']].to_numpy())

data['listing'].x = torch.tensor(listings_scaled, dtype=torch.float32)
data['listing'].y = torch.tensor(np.log(listings_price.to_numpy()), dtype=torch.float32)
data['poi'].x = torch.tensor(pois_scaled, dtype=torch.float32)
data['listing', 'is_near', 'poi'].edge_index = torch.tensor(poi_edge_index, dtype=torch.int64)
data['listing', 'is_near', 'poi'].edge_weight = torch.tensor(poi_edge_weights, dtype=torch.float32)

data

HeteroData(
  listing={
    x=[8528, 13],
    y=[8528],
  },
  poi={ x=[3157, 2] },
  (listing, is_near, poi)={
    edge_index=[2, 1870],
    edge_weight=[1870],
  }
)

In [24]:
data = T.RandomNodeSplit()(data)
data = T.ToUndirected()(data)
data

HeteroData(
  listing={
    x=[8528, 13],
    y=[8528],
    train_mask=[8528],
    val_mask=[8528],
    test_mask=[8528],
  },
  poi={ x=[3157, 2] },
  (listing, is_near, poi)={
    edge_index=[2, 1870],
    edge_weight=[1870],
  },
  (poi, rev_is_near, listing)={
    edge_index=[2, 1870],
    edge_weight=[1870],
  }
)

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt



class HeteroGNN(torch.nn.Module):
  def __init__(self, metadata, hidden_channels, out_channels, num_layers):
    super().__init__()
    self.convs = torch.nn.ModuleList()

    for _ in range(num_layers):
        conv = HeteroConv({edge_type: torch_geometric.nn.conv.GraphConv((-1, -1), hidden_channels) for edge_type in data.edge_types}, aggr='sum')
        self.convs.append(conv)


    self.lin = Linear(hidden_channels, out_channels)

  def forward(self, x_dict, edge_index_dict):
      for conv in self.convs:
        x_dict = conv(x_dict, edge_index_dict, edge_weight_dict=data.edge_weight_dict)
        x_dict = {key: F.leaky_relu(x) for key, x in x_dict.items()}

      return self.lin(x_dict['listing']).clamp(1,8)

model = HeteroGNN(data.metadata(), hidden_channels=1024, out_channels=1, num_layers=2)
optimizer = torch.optim.Adam(model.parameters())

def train():
  model.train()
  optimizer.zero_grad()
  out = model(data.x_dict, data.edge_index_dict)
  mask = data['listing'].train_mask
  target = data['listing'].y[mask].unsqueeze(1)

  loss = F.mse_loss(out[mask], target)
  loss.backward()
  optimizer.step()
  return float(loss)

@torch.no_grad()
def test():
  model.eval()
  pred = model(data.x_dict, data.edge_index_dict)

  accs = []
  for split in ['train_mask', 'val_mask', 'test_mask']:
      mask = data['listing'][split]
      acc = sklearn.metrics.mean_squared_error(data['listing'].y[mask], pred[mask])
      accs.append(float(acc))
      acc = sklearn.metrics.mean_absolute_error(data['listing'].y[mask], pred[mask])
      accs.append(float(acc))
  return accs

epochs_list = []
train_mse_list = []
val_mse_list = []
test_mse_list = []
train_mae_list = []
val_mae_list = []
test_mae_list = []


for epoch in range(1, 100000000):
  loss = train()
  train_mse, train_mae, val_mse, val_mae, test_mse, test_mae = test()

  epochs_list.append(epoch)
  train_mse_list.append(train_mse)
  val_mse_list.append(val_mse)
  test_mse_list.append(test_mse)

  if epoch % 1 == 0:
    plt.plot(epochs_list, train_mse_list , label='Train Loss')
    plt.plot(epochs_list, val_mse_list, label='Test Loss')

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training / Test Losses')
    plt.legend()
    plt.show()


  print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, \n mse  Error: Train: {train_mse:.4f}, ' f'Val: {val_mse:.4f}, Test: {test_mse:.4f}')
  print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, \n mae  Error: Train: {train_mae:.4f}, ' f'Val: {val_mae:.4f}, Test: {test_mae:.4f}')




In [ ]:
import os.path as osp
import time

import torch
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import Data
from sklearn.metrics import mean_squared_error


scaler = StandardScaler()
listings_scaled = scaler.fit_transform(listings.to_numpy())


data = Data(x=torch.tensor(listings_scaled, dtype=torch.float32),
            edge_index=torch.tensor(listings_edge_index, dtype=torch.int64),
            edge_weight=torch.tensor(listings_edge_weights, dtype=torch.float32),
            y=torch.tensor(np.log(listings_price.to_numpy()), dtype=torch.float32)
)
data = T.RandomNodeSplit()(data)

print(data)

train_loader = LinkNeighborLoader(
    data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device, 'x', 'edge_index')
print(device)

model = GraphSAGE(
    data.num_node_features,
    hidden_channels=64,
    num_layers=2,
).to(device)

optimizer = torch.optim.Adam(model.parameters())


def train():
    model.train()

    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        h_src = h[batch.edge_label_index[0]]
        h_dst = h[batch.edge_label_index[1]]
        pred = (h_src * h_dst).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)

    return total_loss / data.num_nodes



@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index, edge_weight=data.edge_weight).cpu()

    print(out.shape)

    clf = LinearRegression()
    clf.fit(out[data.train_mask], data.y[data.train_mask])

    val_acc = mean_squared_error(clf.predict(out[data.val_mask]), data.y[data.val_mask])
    test_acc = mean_squared_error(clf.predict(out[data.test_mask]), data.y[data.test_mask])

    return val_acc, test_acc


for epoch in range(1, 1000):
    loss = train()
    val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')
